## Метод опорных векторов
### Лабораторная работа №2

#### Задание 1
Провести классификацию найденного датасета, методом машины опорных векторов. В формате Markdown писать пояснения. Объяснить почему были выбраны именно такие гиперпараметры, была ли перекрестная проверка, и т.д.

Гиперпараметры:

C: Определяет силу регуляризации в модели SVM.
gamma: Определяет влияние одного тренировочного примера на другие.
kernel: Определяет тип использованного ядра.

In [1]:
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.metrics import accuracy_score, classification_report
from tqdm import tqdm
import pandas as pd

file_path = "Electric_Vehicle_Population_Data.csv"
df = pd.read_csv(file_path)
df.dropna(inplace=True)
# Преобразование категориальной переменной в числовой формат
label_encoder = LabelEncoder()
df['Electric Vehicle Type'] = label_encoder.fit_transform(df['Electric Vehicle Type'])

# Разделение на признаки (X) и целевую переменную (y)
X = df[['Postal Code', 'Model Year', 'Electric Range', 'Base MSRP', 'Legislative District', 'DOL Vehicle ID', '2020 Census Tract']][0:10000]
y = df['Electric Vehicle Type'][0:10000]

# Разделение данных на тренировочный и тестовый наборы
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Масштабирование признаков (нормализация)
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

param_grid = {'C': [0.01, 0.1, 1], 'kernel': ['linear', 'rbf', 'sigmoid'], 'gamma': [0.1, 1, 10]}
# Подбор гиперпараметров с помощью перекрестной проверки
grid = GridSearchCV(SVC(), param_grid, refit=True, verbose=0)
grid.fit(X_train_scaled, y_train)

best_params = grid.best_params_
best_score = grid.best_score_
print("Лучшие гиперпараметры:", grid.best_params_)
best_svc = grid.best_estimator_
y_pred = best_svc.predict(X_test_scaled)
accuracy = accuracy_score(y_test, y_pred)
print(f"Точность модели: {accuracy}")

# Получение отчета о классификации с предупреждениями
report = classification_report(y_test, y_pred, zero_division=1)
print(f'Отчет о классификации:\n{report}')

Лучшие гиперпараметры: {'C': 1, 'gamma': 1, 'kernel': 'rbf'}
Точность модели: 0.984
Отчет о классификации:
              precision    recall  f1-score   support

           0       0.98      1.00      0.99      1600
           1       0.98      0.94      0.96       400

    accuracy                           0.98      2000
   macro avg       0.98      0.97      0.97      2000
weighted avg       0.98      0.98      0.98      2000

